<h1 align="center">Bag-of-Words Representation and Multinomial Naive Bayes Model</h1>
<hr>
<strong>Question 4.4 (Coding) [20 points]</strong>Train a Naive Bayes classifier using all of the data in the training set
( question-4-train-features.csv and question-4-train-labels.csv). Test your classifier on
the test data (question-4-test-features.txt and question-4-test-labels.txt), and report
the testing accuracy as well as how many wrong predictions were made. In estimating the model parameters
use the above MLE estimator. If it arises in your code, dene 0 * log 0 = 0 (note that a * log 0 is as it is,
that is -inf ). In case of ties, you should predict \neutral". Report your test set accuracy. What did your
classifier end up predicting? Why is using the MLE estimate a bad idea in this situation?

In [1]:
# including the required libraries
import numpy  as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
cwd = os.getcwd()

In [2]:
# getting the corresponding data sets.
dataFrameTestFeatures  = pd.read_csv( os.path.join(cwd, "data", "question-4-test-features.csv"),  header=None)
dataFrameTrainFeatures = pd.read_csv( os.path.join(cwd, "data", "question-4-train-features.csv"), header=None)
dataFrameTrainLabels   = pd.read_csv( os.path.join(cwd, "data", "question-4-train-labels.csv"),   header=None)
dataFrameTestLabels    = pd.read_csv( os.path.join(cwd, "data", "question-4-test-labels.csv"),    header=None)

In [3]:
# stacking the test and train labels
npTrain = np.hstack((dataFrameTrainLabels.values, dataFrameTrainFeatures.values))
npTest  = np.hstack((dataFrameTestLabels.values,  dataFrameTestFeatures.values))

In [4]:
# printing the stacked matrices
print('*****************************')
print(npTrain)
print('*****************************')
print(npTest)
print('*****************************')

*****************************
[['neutral' 1 0 ... 0 0 0]
 ['positive' 1 1 ... 0 0 0]
 ['neutral' 1 0 ... 0 0 0]
 ...
 ['negative' 0 0 ... 0 0 0]
 ['negative' 0 0 ... 0 0 0]
 ['positive' 0 0 ... 0 0 0]]
*****************************
[['negative' 0 0 ... 0 0 0]
 ['negative' 0 0 ... 0 0 0]
 ['positive' 0 0 ... 0 0 0]
 ...
 ['neutral' 0 0 ... 0 0 0]
 ['negative' 0 0 ... 0 0 0]
 ['neutral' 0 0 ... 0 0 0]]
*****************************


In [5]:
# getting the corresponding data according to the tweet type.
frameTrain      = pd.DataFrame(npTrain)
npTrainNeutral  = frameTrain[frameTrain[0] == 'neutral'].loc[:, 1:frameTrain.shape[1]].values
npTrainPositive = frameTrain[frameTrain[0] == 'positive'].loc[:, 1:frameTrain.shape[1]].values
npTrainNegative = frameTrain[frameTrain[0] == 'negative'].loc[:, 1:frameTrain.shape[1]].values

# getting the required numbers for calculations.
wordCount = npTrainNeutral.shape[1]
testCount = npTest.shape[0]
trainNeutralCount  = npTrainNeutral.shape[0]
trainPositiveCount = npTrainPositive.shape[0]
trainNegativeCount = npTrainNegative.shape[0]

In [6]:
# printing the required numbers.
print('TEST COUNT: ' + str(testCount))
print('WORD COUNT: ' + str(wordCount))
print('TRAIN NEUTRAL  COUNT: ' + str(trainNeutralCount))
print('TRAIN POSITIVE COUNT: ' + str(trainPositiveCount))
print('TRAIN NEGATIVE COUNT: ' + str(trainNegativeCount))

TEST COUNT: 2928
WORD COUNT: 5722
TRAIN NEUTRAL  COUNT: 2617
TRAIN POSITIVE COUNT: 2004
TRAIN NEGATIVE COUNT: 7091


In [7]:
# printing the train matrices.
print('-------TRAIN NEUTRAL---------')
print(npTrainNeutral)
print('*****************************')

print('-------TRAIN POSITIVE--------')
print(npTrainPositive)
print('*****************************')

print('-------TRAIN NEGATIVE--------')
print(npTrainNegative)
print('*****************************')

-------TRAIN NEUTRAL---------
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
*****************************
-------TRAIN POSITIVE--------
[[1 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
*****************************
-------TRAIN NEGATIVE--------
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
*****************************


In [8]:
# calculating the number of occurrences of word J.
npNumOfWordJOccNeutral  = npTrainNeutral.sum(axis=0)  # Tj,neutral
npNumOfWordJOccPositive = npTrainPositive.sum(axis=0) # Tj,positive
npNumOfWordJOccNegative = npTrainNegative.sum(axis=0) # Tj,negative

# the number of tweets in the training set.
neutralTweetCount  = npTrainNeutral.shape[0]  # N neutral
positiveTweetCount = npTrainPositive.shape[0] # N positive
negativeTweetCount = npTrainNegative.shape[0] # N negative
tweetCount = neutralTweetCount + positiveTweetCount + negativeTweetCount; # N

# estimates the fraction of the tweets with the j-th word of the vocabulary,
probOfOccOfWordJNeutral  = npNumOfWordJOccNeutral  / npNumOfWordJOccNeutral.sum()  # P(Xj | Y = neutral)
probOfOccOfWordJPositive = npNumOfWordJOccPositive / npNumOfWordJOccPositive.sum() # P(Xj | Y = positive)
probOfOccOfWordJNegative = npNumOfWordJOccNegative / npNumOfWordJOccNegative.sum() # P(Xj | Y = negative)

# the probability that any particular tweet will be positive, negative or neutral.
probOfNeutral  = neutralTweetCount  / tweetCount # P(Y = neutral)
probOfPositive = positiveTweetCount / tweetCount # P(Y = positive)
probOfNegative = negativeTweetCount / tweetCount # P(Y = negative)

# converting the content to float.
probOfOccOfWordJNeutral = probOfOccOfWordJNeutral.astype(float)
probOfOccOfWordJPositive = probOfOccOfWordJPositive.astype(float)
probOfOccOfWordJNegative = probOfOccOfWordJNegative.astype(float)

In [9]:
print('--------------------------------------------------------------------------------------------------------------------')
print('Number of Word J Occ in Neutral      : ' + str(npNumOfWordJOccNeutral))
print('Probability of Neutral               : ' + str(probOfNeutral))
print('Number of Neutral Tweets             : ' + str(neutralTweetCount))
print('Probability of Word J Occ in Neutral : ' + str(probOfOccOfWordJNeutral))
print('--------------------------------------------------------------------------------------------------------------------')
print('Number of Word J Occ in Positive     : ' + str(npNumOfWordJOccPositive))
print('Probability of Positive              : ' + str(probOfPositive))
print('Number of Positive Tweets            : ' + str(positiveTweetCount))
print('Probability of Word J Occ in Positive: ' + str(probOfOccOfWordJPositive))
print('--------------------------------------------------------------------------------------------------------------------')
print('Number of Word J Occ in Negative     : ' + str(npNumOfWordJOccNegative))
print('Probability of Negative              : ' + str(probOfNegative))
print('Number of Negative Tweets            : ' + str(negativeTweetCount))
print('Probability of Word J Occ in Negative: ' + str(probOfOccOfWordJNegative))
print('--------------------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------------------
Number of Word J Occ in Neutral      : [175 1 4 ... 0 0 0]
Probability of Neutral               : 0.2234460382513661
Number of Neutral Tweets             : 2617
Probability of Word J Occ in Neutral : [1.08729419e-02 6.21310966e-05 2.48524386e-04 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
--------------------------------------------------------------------------------------------------------------------
Number of Word J Occ in Positive     : [152 2 32 ... 0 0 0]
Probability of Positive              : 0.1711065573770492
Number of Positive Tweets            : 2004
Probability of Word J Occ in Positive: [0.01247333 0.00016412 0.00262596 ... 0.         0.         0.        ]
--------------------------------------------------------------------------------------------------------------------
Number of Word J Occ in Negative     : [189 3 136 ... 0 0 0]
Probability of Neg

In [10]:
# calculating the estimates for every test count.
npTest = dataFrameTestFeatures.values.astype(float)
resultsNeutral  = np.zeros((testCount, 1))
resultsPositive = np.zeros((testCount, 1))
resultsNegative = np.zeros((testCount, 1))

for i in range(testCount):
    tempNeutral  = np.zeros((1, wordCount))
    tempPositive = np.zeros((1, wordCount))
    tempNegative = np.zeros((1, wordCount))
    
    for j in range(wordCount):
        # getting how many times 
        #the word occurred in tweet.
        occOfWordJInTweetI = npTest[i][j]    
        if ((probOfOccOfWordJNeutral[j] != 0 or occOfWordJInTweetI != 0)):
            tempNeutral[0][j]  = occOfWordJInTweetI * np.log(probOfOccOfWordJNeutral[j])
            
        if ((probOfOccOfWordJPositive[j] != 0 or occOfWordJInTweetI != 0)):
            tempPositive[0][j] = occOfWordJInTweetI * np.log(probOfOccOfWordJPositive[j])
            
        if ((probOfOccOfWordJNegative[j] != 0 or occOfWordJInTweetI != 0)):
            tempNegative[0][j] = occOfWordJInTweetI * np.log(probOfOccOfWordJNegative[j])
        
    # calculating MLE
    resultsNeutral[i][0]  = (np.log(probOfNeutral)  + tempNeutral.sum(axis=1))[0]
    resultsPositive[i][0] = (np.log(probOfPositive) + tempPositive.sum(axis=1))[0]
    resultsNegative[i][0] = (np.log(probOfNegative) + tempNegative.sum(axis=1))[0]

C:\Users\FatihPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
C:\Users\FatihPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log
C:\Users\FatihPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


In [11]:
# printing the MLE results for each tweet type.
print('-------Neutral-------')
print(resultsNeutral)
print('---------------------')

print('-------Positive------')
print(resultsPositive)
print('---------------------')

print('-------Negative------')
print(resultsNegative)
print('---------------------')

-------Neutral-------
[[        -inf]
 [        -inf]
 [-24.00093776]
 ...
 [-27.94955759]
 [        -inf]
 [-51.34908695]]
---------------------
-------Positive------
[[        -inf]
 [        -inf]
 [-23.95043791]
 ...
 [-30.4749826 ]
 [        -inf]
 [-56.53544116]]
---------------------
-------Negative------
[[-68.04786297]
 [-80.36426826]
 [-25.97062512]
 ...
 [-29.51003472]
 [-56.51673871]
 [-48.93828622]]
---------------------


In [12]:
# calculating the results for each tweet.
results = []
for i in range(testCount):
    if ((resultsNeutral[i] >= resultsNegative[i]) and (resultsNeutral[i] >= resultsPositive[i])):
        results.append('neutral')
    elif ((resultsNegative[i] >= resultsNeutral[i]) and (resultsNegative[i] >= resultsPositive[i])):
        results.append('negative')
    elif ((resultsPositive[i] >= resultsNeutral[i]) and (resultsPositive[i] > resultsNegative[i])):
        results.append('positive')
        
results = np.array(results).reshape(-1,1)

In [13]:
print('-------Results------')
print(results)
print('--------------------')
print('-----Test Values----')
print(dataFrameTestLabels.values)
print('--------------------')

-------Results------
[['negative']
 ['negative']
 ['positive']
 ...
 ['neutral']
 ['negative']
 ['negative']]
--------------------
-----Test Values----
[['negative']
 ['negative']
 ['positive']
 ...
 ['neutral']
 ['negative']
 ['neutral']]
--------------------


In [14]:
# calculating Accuracy
estimateCount = 0
failureCount  = 0

for i in range(testCount):
    if dataFrameTestLabels.values[i] == results[i]:
        estimateCount = estimateCount + 1
    else:
        failureCount = failureCount + 1

print(estimateCount)
print(failureCount)
accuracy = (estimateCount / (estimateCount + failureCount)) * 100
print('ACCURACY: ' + str(accuracy))

1839
1089
ACCURACY: 62.807377049180324


<h1 align="center">INTERPRETATION OF THE RESULTS</h1>
<hr>
By using this model, an accurracy of <strong>62.807377049180324%</strong> is achieved and even if we were able to predict some data, the accurracy is still not good. Using this MLE is a bad idea since it hasn't got any <strong>smoothing.</strong> Sometimes we get the probability of occurrence of a word as zero because the word hasn't been observed in the dataset. Hence, we get in some cases due to a.log(0) <strong>-inf</strong> which leads to make wrong predictions. If we would use smoothing, then, we could be able to avoid getting a.log(0).